# Seasonal Regression

The electricity demand series shows daily, weekly, and annual oscillations.  At a short time scale, I aim to capture the first two of these.

The approaches I've seen suggest Fourier Series, linear regression, and Seasonal ARIMA.
(I got quite stuck on how to detrend the series in a global fashion.)
I will focus on building models on the last two weeks of data, with the goal of predicting electricity demand based on temperature, time of day, and day of week.

## Hyndman's Multiple Seasonal Exponential Smoothing

This follows Rob Hyndman's approach towards multi-seasonal exponential smoothing.  (This generalizes the apparently well-known Holt-Winters smoothing).
His analysis includes forecasts of electricity generation, based on utility data (from well over 10 years ago).

I chose to follow this model since initial attempts at ARIMA rely on removing the seasonality, and I had hoped to just follow best practice with existing libraries.  Initial naive methods gave complete crap, and failed to remove the seasonal pattern, or even worse imposed one.  An initial attempt at Fourier filtering on over a year of data also left a 

Hyndman also seems to be a known author within the field of econometric time-series forecasting.  

The original model for a variable $y_t$, with seasonal pattern with period $m$ is
\begin{align}
  y_t &= l_{t-1}+b_{t-1} +s_{t-m} +\epsilon_t\\
  l_t &= l_{t-1} + \alpha\epsilon_t\\
  b_t &= b_{t-1} + \beta\epsilon_t\\
  s_{t} = s_{t-m} + \gamma \epsilon_t
\end{align}
where $l_t$, b_t,s_t$ are the level, trend and seasonal patterns respectively.
The noise is Gaussian and obeys
$E[\epsilon_t]=0, E[\epsilon_t\epsilon_s]=\delta_{ts}\sigma^2$, and $\alpha,\beta,\gamma$ are constants between zero and one.  (He notes that $m+2$ estimates must be made for the initial values of the level, trend and seasonal pattern).

Hyndman's model allows multiple seasons, and allows the sub-seasonal terms to be updated more quickly than once per large season.  In utility data, the short season is the daily oscillation, while the longer season comes from the weekly oscillation induced by the work week.  For hourly data, the daily cycle has length $m_1=24$, with the weekly cycle taking $m_2=168$.  The ratio between them is $k=m_2/m_1=7.$  The number of seasonal patterns is $r\le k$.  

(I'm going to change Hyndman's notation to use $\mathbf{I}$ to denote indicator/step functions).
\begin{align}
  y_t &= l_{t-1}+b_{t-1} +\sum_{i=1}^r \mathbf{I}_{t,i}s_{i,t-m_1} +\epsilon_t\\
  s_{i,t} = s_{i,t-m_1} + \sum_{j=1}^r\left(\gamma_{ij}\mathbf{I}_{t,j}\right) \epsilon_t  (i=1,2,\ldots,r)
  l_t &= l_{t-1} + b_{t-1}+\alpha\epsilon_t\\
  b_t &= b_{t-1} + \beta\epsilon_t\\
\end{align}
Here the indicator functions $\mathbf{I}_{t,i}$ are unity if $t$ is in the seasonal pattern $i$, and zero otherwise.  For utility data, this will probably be weekday and holiday/weekend.  Here $\gamma_{ij}$ denotes how much one seasonal pattern is updated based on another---Hyndman proposes a number of restrictions on these parameters.

I will extend this to include an external variables for the deviation above a given temperature, so that $y_t\rightarrow y_t+\tau_p\Theta(T_t-T_p)+\tau_{n}\Theta(T_n-T_t)$.

He suggests using the first four weeks of data to estimate the parameters, by minimizing the squared error of the one-step ahead forecast.  Apparently maximum likelihood estimation was not recommended (10 years ago).

So how to fit the parameters?  A really simple approach would be gradient descent?  Intuitively, the level is the average value, the bias is the average gradient.  The seasonality is the average seasonal pattern.  (This is the dumb STL decomposition used earlier?)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from get_weather_data import convert_isd_to_df, convert_state_isd
from EBA_util import remove_na, avg_extremes

from numpy import pi,e

%matplotlib inline

In [2]:
air_df = pd.read_csv('data/air_code_df.gz')

#Just get the weather station data for cities in Oregon.
df_weather=convert_state_isd(air_df,'OR')
#Select temperature for Portland, OR
msk1=np.array(df_weather['city']=='Portland')
msk2=np.array(df_weather['state']=='OR')

df_pdx_weath=df_weather.loc[msk1&msk2]

#get electricity data for Portland General Electric
df_eba=pd.read_csv('data/EBA_time.gz',index_col=0,parse_dates=True)
msk=df_eba.columns.str.contains('Portland')
df_pdx=df_eba.loc[:,msk]



done with Mahlon Sweet Field


done with Salem Municipal Airport/McNary Field


done with Portland International Airport


In [3]:
dem=df_pdx.iloc[:,0]
#Make a combined Portland Dataframe for demand vs weather.
df_joint=pd.DataFrame(dem)
df_joint=df_joint.join(df_pdx_weath)
temp=df_joint['Temp']
df_joint['TempShift']=150+abs(temp-150)
df_joint=df_joint.rename(columns={df_joint.columns[0]:'Demand'})


In [4]:
#clean up data, remove NA
dem = remove_na(dem)
dem = avg_extremes(dem)
temp = avg_extremes(remove_na(temp))

Number of extreme values 0. Number of zero values 148


Number of NA values 56


Number of extreme values 1. Number of zero values 3


Number of NA values 156


In [17]:
#from EBA_seasonal import multiseasonal as ms
#Class for Multiseasonal model.
class ms:
    def fit_init_params(y,ninit=4*24*7):
         """fit_init_params(y)
         Fits initial parameters for Hyndman's multi-seasonal model to
         hourly electricity data.
         (My guess on how to do this, similar to naive STL method used in 
         statstools.timeseries)
         Finds level, bias and seasonal patterns based on first 4 weeks of data.  
         """
         ysub = y[0:ninit]
         yval = ysub.values
         ##average value
         l = np.mean(yval)
         ##average shift
         b = np.mean(np.diff(yval))
         ##remove mean pattern, subtract off level, and linear trend.
         ysub = ysub-l-b*np.arange(ninit)
         #mean seasonal pattern.
         #second seasonal pattern is for weekends, with days
         #Saturday/Sunday have dayofweek equal to 5 and 6.
         #make a mask to select out weekends.
         s2 = ysub.index.dayofweek >=5
         #select out weekends, and regular days. 
         y_end = ysub[s2]
         y_week=ysub[~s2]
         n1 = int(len(y_week)/24)
         n2 = int(len(y_end)/24)
         s = np.zeros((2,24))
         print(n1,n2)
         for n in range(n1):
              s[0,:] = s[0,:]+y_week[n*24:(n+1)*24]/n1
         for n in range(n2):
              s[1,:] = s[1,:]+y_end[n*24:(n+1)*24]/n2

         return l, b, s

         # def predict_stl(l,b,s,timeIndex):
         #          """predict_stl(l,b,s,timeIndex)
         #          Predicts STL time-series for a fixed set of parameters.
         #          Not useful.
         #          """
         #          # n1 = int(sum(~msk)/24)         
         #          # n2 = int(sum(msk)/24)
         #          #Use fact that first sub-season is weekdays in first row.
         #          #Use integer conversion of true/false to 0/1.
         #          #Then use fact that seasonal patterns are 24 hours long to select right hour.
         #          #find weekend/weekedays.  
         #          msk=timeIndex.dayofweek>=5
         #          trend=l+b*np.arange(len(timeIndex))
         #          pred=trend+s[msk.astype(int),timeIndex.hour.values]
         #          return pred

    def STL_step(l,b,s,alpha,beta,gamma,yhat,ypred,t):
         """STL_step
         Updates time parameters based on differences between predicted 
         and measured.  
         (Not fixing the model parameters for update strength!)
         Work in Progress
         """
         eps=(yhat-ypred)
         #find seasonal patterns.  
         m1 = t.hour                     
         mr = int(t.dayofweek>=5)
         ynew = l + b + s[mr,m1]
         l = l+b+alpha*eps
         b = b+beta*eps
         #update row, and hour
         ds =np.dot(gamma,np.array([~mr,mr]))*eps
         s[:,m1] += ds
         # global icount
         # icount +=1
         # if (icount%(24*7)==0):
         #          print(t)
         #          print(alpha*eps,beta*eps,ds,'\n')
         return l,b,s,ynew

    def predict_STL(y,alpha,beta,gamma):
         """predict_STL
         Generates initial parameters, and then predicts remainder
         of series for input data y.  
         """
     
         ninit=4*24*7
         l,b,s=fit_init_params(y,ninit=4*24*7)
         t0=y.index[ninit+1]
         m1 = t0.dayofweek>=5
         m2 = t0.hour
         
         ypred = l+b*ninit+s[int(m1),m2]
         ytot = np.zeros(len(y))
         print(s[int(m1),m2])
         for i in range(ninit,len(y)):
             l,b,s,ypred = STL_step(l,b,s,
                            alpha,beta,gamma,
                              y[i],ypred,y.index[i])
         ytot[i]=ypred
         # if i%(24*7) ==0:
         #         print("l: {} b: {}\n".format(str(l),str(b)))
         #         print(s,"\n")
         ytot=pd.Series(ytot,index=y.index)         
         return ytot,l,b,s


In [282]:
%pdb

Automatic pdb calling has been turned OFF


In [134]:
dem_sub = dem[0:24*7*20]
alpha=0.01
beta=0.01
gamma=0.1*np.array([[1,0.1],[0.1,1]])
global icount
icount=0
ytot,l,b,s=ms.predict_STL(dem_sub,alpha,beta,gamma)
per='2016-01'
plt.plot(ytot,'b',label='Predicted')
plt.plot(dem_sub,'r',label='Actual')
plt.legend()
plt.show()

20 8
996.490836349


In [19]:
plt.plot(np.log(np.abs(ytot)))
plt.show()

/home/jonathan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [454]:
l,b,s=ms.fit_init_params(dem_sub,ninit=4*24*7)
plt.plot(s.T)
plt.show()

20 8


# Thresholded Temperature Model

Let's just try to build a linear model for the temperature on top of this.
I'll assume that heating/cooling might have different coefficients, so the temperature component of the model at time $t$ is
\begin{equation}
D_t =  a_0+ a_+[T_t-T_{+}]_{+} + a_-[T_{-}-T_t]_{+},
\end{equation}
where $[f]_+=f$ if $f>0$, and is zero otherwise.  If we optimize the mean square
error, then the components can be found by solving for the values that minimize the derivatives.
If criteria is mean square error, then can solve directly for parameters
$J = sum_t[\hat{D}_t-D_t)^2$, where $\hat{D}_t$ is the true value.
We must solve $\partial J/\partial \alpha = 0 \rightarrow \sum_t \frac{\partial D_t}{\partial\alpha}(\hat{D}_t-D_t) $

Those conditions are
\begin{align}
    \sum_t(\hat{D}_t-D_t)=0  \qquad (a_0)\\
    \sum_{t\in T_\pm}[T_t-T_\pm]_\pm(\hat{D}_t-D_t)=0  \qquad (a_\pm)\\
    \sum_{t\in T_\pm}(\pm a_{\pm})(\hat{D}_t-D_t)=0  \qquad (T_\pm)\\
\end{align}


(First OOP class - absolutely ridiculous)  Will just use a pandas Dataframe - want a named set of numbers, with defined operations.  Already here.  

In [34]:
#Experimenting with OOP for making "vector" of parameters with named labels.
#Feels daft - if there's a less stupid way, I'll try to fix this.  (Just use a Dataframe!?)
#Initial attempts at getting smarter initialization (with arguments to make a dict just returned empty.
# class param_vec(dict):
#     """Class for model parameters.
#     Stores parameters in dict, with arithmatic operations.
#     """
#     #Define elementwise subtraction/addition
#     def __add__(self,x):
#         y = self.copy()
#         if isinstance(x,(int,float)):
#             for i,v in self.items():
#                 y[i]=self[i]+x
#         else:
#             for i,v in self.items():
#                 y[i]=self[i]+x[i]
#         return param_vec(y)

#     #Define elementwise subtraction/addition
#     def __radd__(self,x):
#         return param_vec.__add__(self,x)
    
#     #Define elementwise subtraction.
#     def __sub__(self,x):
#         y=self.copy()
#         if isinstance(x,(int,float)):
#             for i,v in self.items():
#                 y[i]=v-x
#         else:        
#             for i,v in self.items():
#                 y[i]=v-x[i]
#         return param_vec(y)
                
#     #Define elementwise subtraction/addition
#     def __rsub__(self,x):
#         return param_vec.__sub__(self,x)
                
#     #Define elementwise subtraction.
#     def __truediv__(self,x):
#         y=self.copy()
#         if isinstance(x,(int,float)):
#             for i,v in self.items():
#                 y[i]=v/x
#         else:        
#             for i,v in self.items():
#                 y[i]=v/x[i]
#         return param_vec(y)
                
#     #Define elementwise subtraction.
#     def __mul__(self,x):
#         y=self.copy()
#         if isinstance(x,(int,float)):
#             for i,v in self.items():
#                 y[i]=v*x
#         else:        
#             for i,v in self.items():
#                 y[i]=v*x[i]
#         return param_vec(y)
                
#     #Define elementwise subtraction.
#     def __rmul__(self,x):
#         return param_vec.__mul__(self,x)

def param_vec(names,vals):
    p=pd.Series(vals,index=names)
    return p

#might extend to contain all of the modelling stuff?          

In [94]:
dem_sub  = dem[0:24*7*1]
temp_sub = temp[0:24*7*1]

D=dem_sub
T=temp_sub
Dr = np.max(D)-np.min(D)
Tr = np.max(T)-np.min(T)

pnames=['a0','ap','an','Tp','Tn']
pval=[np.mean(D),0.5*Dr/Tr,0.5*Dr/Tr,200,100]

Tmodel=just_temp_model(names=pnames,vals=pval)



In [135]:
class just_temp_model:
    def __init__(self,names=[],vals=[]):
        self.param= pd.Series(vals,index=names)

    def temp_model(self,T):
        """temp_model(self,T)
        Tries to fit linear model for electricity demand to temperature.
        Initially tried to allow thresholding, and different slopes for heating/cooling.  
        Will now just try simpler linear model a_p|T-T_p|.
        """ 
        m1 = T>self.param['Tp']
        m2 = T<self.param['Tn']
        y=np.zeros(T.shape)
        y[m1] = (T[m1]-self.param['Tp'])*self.param['ap']
        y[m2] = (self.param['Tn']-T[m2])*self.param['an']
        y=y+self.param['a0']
        #y = param['a0']+ param['ap']*np.abs(T-param['Tp'])
        y=pd.Series(y,name='Predicted Demand',index=T.index)
        return y

    def temp_model_grad(self,D,Dhat,T):
        """temp_model_grad
        Compute gradients of model w.r.t. parameters.
        Assumes loss-function is mean-square.
        Dhat - measured demand
        D    - predicted demand
        T    - measured temperature
        """
        m1 = T>self.param['Tp']
        m2 = T<self.param['Tn']
        Nt = len(T)
        Derr=D-Dhat
        #initialize with zeros
        dparam=param_vec(self.param.keys(),np.zeros(len(self.param)))
        dparam['a0'] = np.sum(Derr)/Nt
        #Single model
        # dparam['ap'] = np.sum( np.abs(T-param['Tp'])*Derr)/Nt
        # dparam['Tp'] = -np.sum(np.sign(T-param['Tp'])*param['ap']*Derr)/Nt
        #Double thresholded model
        dparam['ap'] = np.sum( np.abs(T[m1]-self.param['Tp'])  \
                     *(Derr[m1]))/Nt
        dparam['an'] = np.sum( (self.param['Tn']-T[m2])*(Derr[m2]))/Nt
        dparam['Tp'] = -self.param['ap']*np.sum(Derr[m1])/Nt
        dparam['Tn'] =  self.param['an']*np.sum(Derr[m2])/Nt    
        return dparam
    
    def param_fit(self,Dhat,T,alpha=0.1,rtol=1E-4,nmax=200):
        """Try to fit linear threshold model of demand to temperature.
            D - demand data
            T - temperature data
            Fits model of form:
            D ~ a_0+ a_p[T-T_p]_+ + a_n[T_n-T]_+,
            where [f]_+ =f for f>0, and 0 otherwise.

            Just use simple gradient descent to fit the model.
        """
        #make parameter estimates
        Dr = np.max(Dhat)-np.min(Dhat)
        Tr = np.max(T)-np.min(T)
        param_names=['a0','ap','an','Tp','Tn']
        param_vals=[np.mean(Dhat), 0.5*Dr/Tr, 0.5*Dr/Tr,
        np.mean(T), np.mean(T)]
        self.param=param_vec(param_names,param_vals)
        Dpred = self.temp_model(T)
        J=np.sum((Dpred-Dhat)**2)/len(Dhat)
        print('Init cost',J)
        plot_pred(Dpred,Dhat,10*T)
        print('Param:',self.param,"\n")    

        Ni=0
        for i in range(nmax):
            dparam=self.temp_model_grad(Dpred,Dhat,T)
            self.param=self.param-alpha*dparam
            Dpred=self.temp_model(T)
            J2=J        
            J=np.sum((Dpred-Dhat)**2)/len(Dhat)
            err_change=abs(1-J2/J)
            Ni+=1
            if (err_change<rtol):
               print("Hit tolerance {} at iter {}".format(
               err_change,Ni))
               plot_pred(Dpred,Dhat,T)                      
               return param,Dpred
            if(Ni%100==0):
                print("Cost, Old Cost = {},{}".format(J,J2))
                print('Param:',self.param)
                print('Param_grad:',dparam)
                print("Mean param Change {} at iter {}".format(err_change,Ni))
                plot_pred(Dpred,Dhat,T)
        print("Failed to hit tolerance after {} iter\n".format(iter))
        print("Cost:",J,J2)
        return param, Dpred 

    def grad_check(self,Dhat,T):
        """grad_check(Dhat,T)
        Check numerical gradients against finite difference.
            D - demand data
            T - temperature data
        """
        #make parameter estimates
        Dr = np.max(Dhat)-np.min(Dhat)
        Tr = np.max(T)-np.min(T)
        param_names=['a0','ap','an','Tp','Tn']
        param_vals=1000*np.random.random(size=5)
        self.param=param_vec(param_names,param_vals)
        print(self.param)
        Dpred = self.temp_model(T)    
        dparam=self.temp_model_grad(Dpred,Dhat,T)    
        J=0.5*np.sum((Dpred-Dhat)**2)/len(Dhat)
        eps=.01
        param2 = self.param.copy()
        for name,val in param2.items():
            self.param = param2.copy()
            self.param[name]=val+eps
            Dpred = self.temp_model(T)
            J2=0.5*np.sum((Dpred-Dhat)**2)/len(Dhat)
            numgrad = (J2-J)/eps
            print(name,numgrad,dparam[name],self.param[name])

#End of temp_model class.
#Needed to scale the data to get decent answers.  

def scale_data(D,T):
    """scale_data
    Scales both demand and temperature to have zero mean, unit standard deviation.
    Returns scaled data, as well as means, and standard deviations.
    """
    scaling_df=pd.DataFrame(columns=['dem','temp'],index=['mu','std'])
    scaling_df.loc['mu','dem']=D.mean()
    scaling_df.loc['std','dem']=D.std()
    scaling_df.loc['mu','temp']=T.mean()
    scaling_df.loc['std','temp']=T.std()
    T_scale=(T-T.mean())/T.std()
    D_scale=(D-D.mean())/D.std()
    return D_scale,T_scale,scaling_df

def plot_pred(Dpred,D,T):
    """make plot to compare fitted parameters"""
    plt.plot(T,'r',label='temp')    
    plt.plot(D,'g',label='obs')
    plt.plot(Dpred,'b',label='pred')
    plt.legend()
    plt.show()


In [133]:
dem_sub  = dem[0:20]
temp_sub = temp[0:20]
#D,T,scale_df=scale_data(dem_sub,temp_sub)
Tmodel=just_temp_model(names=pnames,vals=pval)
Tmodel.grad_check(dem_sub,temp_sub)

a0    321.518791
ap    995.282815
an    681.873253
Tp    743.912988
Tn    201.516651
dtype: float64
a0 -313657.0816040039 -313657.090282 321.528790544
ap 151737230.07202148 0.0 995.292814876
an 0.0 -12501.1763538 681.883252594
Tp 311551992.6437378 -0.0 743.922987577
Tn 0.0 -430751.896026 201.526651372


In [ ]:
# Fourier Series

Another approach that I've seen to seasonality is to just use a Fourier series.
This is similar to an approach I was using based on trying to filter the Fourier tranformed data.
This method tries to fit annual, weekly and daily oscillations to the data. 


In [56]:
#Now to do some simple Fourier Series fitting too.
class fourier_model:
    def total_fourier_series(self,D,n_max=[2,4,4]):
        """fourier_series
        Fits pandas time series D, with Fourier series. 
        Uses Pandas DateTimeIndex for times.
        Produces fourier series with annual, daily and weekly oscillations to fourier series.
        Computes coefficients, and then series.  Returns both
        D - demand (values to be fitted)
        T - DatetimeIndex
        n_max - maximum number of coefficients

        Note:Misses Holidays.
        """
        T=D.index
        T_dayofyear = T.dayofyear.values
        T_dayofweek = T.dayofweek.values
        T_hour = T.hour.values
        Tfit = [T_dayofyear/365,
        (T_dayofweek*24+T_hour)/168,
        T_hour/24]
        periods=[365,168,24]
        Nt = len(D)
        ftot = np.zeros(Nt)
        coeff=[[np.sum(D)/Nt,0]]    
        for i in range(3):
            if n_max[i]>0:
                 ci    = self.fit_fourier_series(D,Tfit[i],n_max[i])
                 ftot += self.fourier_series(ci,Tfit[i])             
            else:
                ci=None
            coeff.append(ci)
        #add on constant    
        ftot+= coeff[0][0]
        ftot=pd.Series(ftot,index=T)         
        return ftot,coeff

    def fit_fourier_series(self,D,T,nmax):
        """Fits the Fourier series to data D, on times T,
        and returns parameters.
        """
        Nt = len(D)
        #initial zero coefficients
        coeff=[]
        for n in range(1,nmax+1):
            an= 2*np.sum(np.cos(2*pi*n*T)*D)/Nt
            bn= 2*np.sum(np.sin(2*pi*n*T)*D)/Nt
            coeff.append([an,bn])
        return coeff                       

    def fourier_series(self,coeff,T):
        """fourier_series
        Make simple Fourier series with specified coefficients, 
        over time T. 
        T assumed to be in range [0,1).
        """
        i=1
        nmax=len(coeff)
        f=np.zeros(T.shape)
        #need a +1 somewhere due to 0-indexing, and not including constant
        for n in range(nmax):
            an, bn = coeff[n]
            f += an*np.cos(2*pi*(n+1)*T)+bn*np.sin(2*pi*(n+1)*T)
        return f

In [88]:
%pdb

Automatic pdb calling has been turned OFF


In [57]:
dem_sub  = dem[0:24*7*4]
temp_sub = 10*temp[0:24*7*4]

nmax = [0,4,4]

Df, Dcoeff = fourier_model.total_fourier_series(dem_sub,nmax)
plot_pred(Df,dem_sub,temp_sub)

AttributeError: 'builtin_function_or_method' object has no attribute 'dayofyear'

In [294]:
%pdb

Automatic pdb calling has been turned ON


In [221]:
dem_scale,temp_scale,scale_df=scale_data(dem_sub,temp_sub)
plt.figure()
param,Dpred=param_fit(dem_scale,temp_scale)

Init cost 1.0459630818455434


Param: {'a0': 2.0403652312679588e-16, 'ap': 0.46708059154688253, 'an': 0.46708059154688253, 'Tp': -7.599740942374584e-18, 'Tn': -7.599740942374584e-18} 



Cost, Old Cost = 0.2195143515128714,0.2197024082319617
Param: {'a0': -0.35311812807232357, 'ap': 0.81915796361757598, 'an': -0.55496339513011805, 'Tp': -0.58384597356876855, 'Tn': -0.10167281635891282}
Param_grad: {'a0': -0.0075882356519908645, 'ap': 0.017650932004730162, 'an': 0.021595530350529406, 'Tp': 0.028091509562537323, 'Tn': 0.0089551371980910923}
Mean param Change 0.0008566944156234158 at iter 100


Cost, Old Cost = 0.21336496095305546,0.21339646918507224
Param: {'a0': -0.34032674885453923, 'ap': 0.74556786333968239, 'an': -0.68420417537364742, 'Tp': -0.70298109102774575, 'Tn': -0.19052066162488607}
Param_grad: {'a0': 0.0002134943921533122, 'ap': 0.0029321058537893157, 'an': 0.009966485139349059, 'Tp': 0.0066009118309116226, 'Tn': 0.0095204945162540022}
Mean param Change 0.00014767294440498624 at iter 200


Failed to hit tolerance after <built-in function iter> iter

Cost: 0.21336496095305546 0.21339646918507224


In [219]:
l,b,s=fit_init_params(dem_sub)

pred=predict_stl(l,b,s,dem_sub.index)

20 8


In [220]:
plt.plot(dem_sub.values,'b',temp_sub.values,'r',pred,'k')
plt.show()

1

Rambling Time!

From a Kalman filter perspective, I think that sometimes the error/innovation terms can be written as $\epsilon_t = y_t-\hat{y}_t$, where $y_t$ is the actual value, and $\hat{y}_t$ is the output of the model with no noise.  The innovation process, then gives a rule for updating (the set of parameters $\alpha,\beta,\Gamma, l, b, s_{i,t}$) how to change in the 

In [35]:
%pdb

Automatic pdb calling has been turned OFF
